# **Tiền xử lý dữ liệu**

In [1]:
import pandas as pd

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_10k.csv")

In [2]:
# Nhóm số lượng dựa theo cột rating
df.groupby('rating').describe()

comment                                                               
         count unique                                                top freq
rating                                                                       
1         9826   9823                                     kém chất lượng    2
5         9727   9727   Hàng đẹp hơn mong đợi , chất lượng tốt phù hợ...    1

In [3]:
df.head(10)

,rating,comment
0,1,"Đóng gói cận thận, độ chống nhìn trộm t/bình, ..."
1,1,"sản phầm chưa dùng đã hỏng, Sản phầm không đún..."
2,1,"Sản phẩm giao chưa đúng theo yêu cầu, Shop khô..."
3,1,"trừ 1 sao vì đóng gói ẩu, chỉ lồng duy nhất tú..."
4,1,"*Hãy xem đánh giá 1⭐, Rất bức xúc😡Không đáng 1..."
5,1,. Loa mới dùng 1 ngày là hông nghe tiếng nữa💁‍...
6,1,.thiếu 1. Số dụng củ
7,1,?????? Nhầm hàng shop ơi cái này là cái gì :D ...
8,1,??shop đặt dây 20w về dây 27w
9,1,_QUÁ TỆ_ Khuyên thật các bạn đừng mua hàng sho...


In [4]:
import re

# Làm sạch văn bản
def clean_text(text):

    #Loại bỏ email
    text = ' '.join([i for i in text.split() if '@' not in i])

    #Loại bỏ website
    text = re.sub('http[s]?://\S+', ' ', text)

    #Loại bỏ tất cả các dấu câu và ký tự không phải chữ cái và số.
    text = re.sub(r'[^\w\s]', ' ', text)

    #oại bỏ tất cả các khoảng trắng nhiều hơn 1 và thay thế chúng bằng một khoảng trắng duy nhất.
    text = re.sub('\s+', ' ', text)

    return text

df["clean_comment"] = df.comment.apply(lambda x: clean_text(x))

In [5]:
# bỏ cột "comment" gốc
df = df.drop(['comment'], axis=1)

# Đổi tên cột "clean_comment" thành "comment"
df.rename(columns = {'clean_comment':'comment'}, inplace = True)

df.head(10)

,rating,comment
0,1,Đóng gói cận thận độ chống nhìn trộm t bình Độ...
1,1,sản phầm chưa dùng đã hỏng Sản phầm không đúng...
2,1,Sản phẩm giao chưa đúng theo yêu cầu Shop khôn...
3,1,trừ 1 sao vì đóng gói ẩu chỉ lồng duy nhất túi...
4,1,Hãy xem đánh giá 1 Rất bức xúc Không đáng 1 s...
5,1,Loa mới dùng 1 ngày là hông nghe tiếng nữa Đú...
6,1,thiếu 1 Số dụng củ
7,1,Nhầm hàng shop ơi cái này là cái gì D mất 65k...
8,1,shop đặt dây 20w về dây 27w
9,1,_QUÁ TỆ_ Khuyên thật các bạn đừng mua hàng sho...


In [6]:
import re

# Thay thế "\n\n" bằng ". " và "\n" bằng " "
df['comment'] = df['comment'].str.replace("\n\n", ", ")
df['comment'] = df['comment'].str.replace("\n", ", ")

# Kiểm tra độ dài và xoá các dòng có ít hơn 15 ký tự trong cột "comment"
min_length = 15
df = df[df['comment'].apply(lambda x: len(str(x)) >= min_length)]

# Kiểm tra và xóa các dòng có văn bản trùng lặp trong cột "comment"
df = df.copy()
df.drop_duplicates(subset="comment", keep="first", inplace=True)

def remove_rows_with_keywords(comment):
    keywords = ["[TB]", "[QC]", "(QC)", "(TB)", "(AD)", "quý khách", "quy khach"]

    for keyword in keywords:
        if re.search(re.escape(keyword), str(comment), flags=re.IGNORECASE):
            return None  # Trả về None để xóa hàng

    return comment

# Áp dụng hàm cho cột "comment"
df['comment'] = df['comment'].apply(remove_rows_with_keywords)

# Loại bỏ các hàng có giá trị là None
df = df.dropna(subset=['comment'])

# Chuyển đổi giá trị trong cột "rating"
df['rating'] = df['rating'].replace({1: 'negative', 5: 'positive'})

# Lưu lại DataFrame vào file CSV mới
df.to_csv("data_processed.csv", index=False, encoding='utf-8-sig')

In [7]:
# Đọc dữ liệu từ file CSV đã xử lý
comment_df = pd.read_csv("data_processed.csv")

# Nhóm số lượng dựa theo cột rating
comment_df.groupby('rating').describe()

comment                                                            \
           count unique                                                top   
rating                                                                       
negative    9672   9672  Đóng gói cận thận độ chống nhìn trộm t bình Độ...   
positive    9672   9672  Hàng đẹp hơn mong đợi chất lượng tốt phù hợp v...   

               
         freq  
rating         
negative    1  
positive    1

In [8]:
from stop_words import get_stop_words
from underthesea import word_tokenize
from nltk.stem.porter import PorterStemmer

stopwords_vi = get_stop_words('vi')  # Danh sách stopwords tiếng Việt
list_word = []
ps = PorterStemmer()

for i in range(len(comment_df)):
        review = comment_df['comment'][i].lower()

        # Chuyển đổi thành danh sách từ sử dụng word_tokenize
        words = word_tokenize(review, format="text")

        # Tiếp tục với các bước xử lý tiếp theo nếu cần
        words = [ps.stem(word) for word in words.split() if word not in stopwords_vi]
        review = ' '.join(words)
        list_word.append(review)

In [9]:
# Chuyển đổi dữ liệu văn bản sang dạng vector bằng TFIDF VECTORIZER
from sklearn.feature_extraction.text import TfidfVectorizer
cv=TfidfVectorizer(max_features=500)
x=cv.fit_transform(list_word).toarray()

In [10]:
y=df['rating']

# **Xây dựng mô hình huấn luyện**

In [11]:
# Chia dữ liệu thành 2 tập train, test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [12]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)

In [13]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy_score: ', accuracy_score(y_test, y_pred))

[[1740  148]
 [ 145 1836]]
              precision    recall  f1-score   support

    negative       0.92      0.92      0.92      1888
    positive       0.93      0.93      0.93      1981

    accuracy                           0.92      3869
   macro avg       0.92      0.92      0.92      3869
weighted avg       0.92      0.92      0.92      3869

accuracy_score:  0.9242698371672267


In [15]:
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

# Hàm tiền xử lý cho một đoạn văn bản
def preprocess_text(text):
    # Thực hiện các bước tiền xử lý tương tự như trong quá trình huấn luyện
    words = word_tokenize(text, format="text")
    words = [ps.stem(word) for word in words.split() if word not in stopwords_vi]
    return ' '.join(words)

# Chuẩn bị dữ liệu đầu vào mới
# new_text = "Sản phẩm tốt nên mua, giao hàng nhanh"
new_text = "hàng kém chất lượng, đánh giá 1 sao"
preprocessed_text = preprocess_text(new_text)

# Sử dụng TfidfVectorizer để chuyển đổi đoạn văn bản thành vectơ TF-IDF
new_text_vector = cv.transform([preprocessed_text]).toarray()

# Dự đoán bằng mô hình đã huấn luyện
prediction = clf.predict(new_text_vector)

# In kết quả dự đoán
print("Đánh giá trên thuộc loại:", prediction)

Đánh giá trên thuộc loại: ['negative']
